# Np's and Mesogens, Temperature Ramp 5.6 -> 5.8

In [6]:
#-----Import essential packages.

from __future__ import division
import hoomd
import hoomd.md

In [7]:
#-----Define important variables.

p_max = 1.8;
t_0   = 5.6; 
t_1   = 5.8;
steps_ramp = 5e4;

In [8]:
#-----Declare the file from wich we'll extract the configuration of the system.

ramp_file = "T_CM&NP_" + str(t_0) + "_P_" + str(p_max) + "_ramp.gsd"

In [9]:
#-----Declare a simulation context.

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Initialize from a .gsd file.

system = hoomd.init.read_gsd(ramp_file, frame = -1);

#-----Define each mesogen in the local frame of reference of each rigid body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types = ['A']*8,
                positions = [(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Create the mesogens.

rigid.create_bodies();

#-----Define the potential energy.

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode = 'shift')

#-----Declare interactions.

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#----Integrate using an NPT integrator.

temp = hoomd.variant.linear_interp(points = [(0,t_0), (steps_ramp, t_1)]);

npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = temp, tau = 2.0, tauP = 3.0, P = p_max);
npt.randomize_velocities(seed=42)

#-----Save .log y .gsd data.

log_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True); 

notice(2): Group "all" created containing 1004 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "Nano_Particles" created containing 4 particles
notice(2): Group "rigid_center" created containing 1000 particles
notice(2): Group "NP_Mes" created containing 1004 particles


In [10]:
#-----Run the simulation.

hoomd.run(steps_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9004
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 657139 / 707000 | TPS 13.753 | ETA 01:00:25
Time 00:00:20 | Step 657338 / 707000 | TPS 19.878 | ETA 00:41:38
Time 00:00:30 | Step 657523 / 707000 | TPS 18.4897 | ETA 00:44:35
Time 00:00:40 | Step 657706 / 707000 | TPS 18.1747 | ETA 00:45:12
Time 00:00:50 | Step 657894 / 707000 | TPS 18.6074 | ETA 00:43:59
Time 00:01:00 | Step 658072 / 707000 | TPS 17.6819 | ETA 00:46:07
Time 00:01:10 | Step 658250 / 707000 | TPS 17.6332 | ETA 00:46:04
Time 00:01:20 | Step 658434 / 707000 | TPS 18.3861 | ETA 00:44:01
Time 00:01:30 | Step 658608 / 707000 | TPS 17.3996 | ETA 00:46:21
Time 00:01:40 | Step 658790 / 707000 | TPS 18.1958 | ETA 00:44:09
Time 00:01:50 | Step 658977 / 707000 | TPS 18.6295 | ETA 00:42:57
Time 00:02:00 | Step 659160 / 707

In [11]:
#-----Get density.

system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x1191e44e0>),
             ('particles', <hoomd.data.particle_data at 0x1192d8be0>),
             ('number_density', 0.6513083463921334),
             ('bonds', <hoomd.data.bond_data at 0x1192d8c88>),
             ('angles', <hoomd.data.angle_data at 0x1192d8cf8>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x1192d8d30>),
             ('impropers', <hoomd.data.dihedral_data at 0x1192d8da0>),
             ('constraints', <hoomd.data.constraint_data at 0x1192d8e10>),
             ('pairs', <hoomd.data.bond_data at 0x1192d8e80>),
             ('timestep', 707000)])

In [12]:
#-----Get volume.

system.box.get_volume()

13824.48121519843